<a href="https://colab.research.google.com/github/Yaswanthyarra/GAN-for-microexpression-augmentation/blob/main/GoogleNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
dataset_dir = '/content/drive/My Drive/CASMEII/Final_Apex_Frames_Segregated'
augmented_dir='/content/drive/My Drive/CASMEII/Augmented_Apex_Frames'
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split


dataset_dir = '/content/drive/My Drive/CASMEII/Final_Apex_Frames_Segregated'

label_dict = {'happiness': 0, 'disgust': 1, 'repression': 2, 'surprise': 3, 'fear': 4, 'others': 6, 'sadness': 5}

expression_images = []
expression_labels = []

for class_name, label in label_dict.items():
    class_dir = os.path.join(dataset_dir, class_name)
    images = os.listdir(class_dir)

    for image_file in images:
        image_path = os.path.join(class_dir, image_file)
        image = cv2.imread(image_path)  # Load the image using OpenCV
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (224,224))
        if label==1:
          expression_images.append(image)
          expression_labels.append(label)
        if label==6:
          expression_images.append(image)
          expression_labels.append(label)



        expression_images.append(image)
        expression_labels.append(label)

augmented_dir='/content/drive/My Drive/CASMEII/Augmented_Apex_Frames'
label_dict = {'happiness': 0, 'repression': 2, 'surprise': 3, 'fear': 4, 'sadness': 5}

for class_name, label in label_dict.items():
    class_dir = os.path.join(augmented_dir, class_name)
    images = os.listdir(class_dir)

    for image_file in images:
        image_path = os.path.join(class_dir, image_file)
        image = cv2.imread(image_path)  # Load the image using OpenCV
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        if label==0:
          expression_images.append(image)
          expression_labels.append(label)

        expression_images.append(image)
        expression_labels.append(label)


        expression_images.append(image)
        expression_labels.append(label)


Mounted at /content/drive


In [2]:
from PIL import Image
import numpy as np

# Assuming your expression_images array has shape (num_images, height, width, channels)

# Resize images to a consistent shape
target_shape = (224, 224)

resized_images = []
for img in expression_images:
    # Ensure the image has the correct shape and data type
    img = np.squeeze(img)  # Remove singleton dimensions
    img = img.astype('uint8')  # Convert to unsigned integer type
    img = Image.fromarray(img)

    # Resize the image
    img = img.resize(target_shape, Image.ANTIALIAS)
    resized_images.append(np.array(img))

expression_images = np.stack(resized_images)

# Ensure consistent data type
expression_images = expression_images.astype('float32')

# Rest of your code...


<ipython-input-2-aaf2c4bf8ca3>:17: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize(target_shape, Image.ANTIALIAS)


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# Convert lists to NumPy arrays
expression_images = np.array(expression_images)
expression_labels = np.array(expression_labels)

# Check the shape of the image data and labels
print("Expression Images Shape:", expression_images.shape)
print("Expression Labels Shape:", expression_labels.shape)

# Obtain the number of class labels
num_classes = len(np.unique(expression_labels))

# Convert labels to one-hot encoding
expression_labels_one_hot = to_categorical(expression_labels, num_classes=num_classes)

# Stratified Splitting of data into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(
    expression_images, expression_labels_one_hot, test_size=0.2, stratify=expression_labels, random_state=42
)

# Load the InceptionV3 model pre-trained on ImageNet data
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Create a sequential model
model = Sequential()

# Add the InceptionV3 base model
model.add(base_model)

# Add a global average pooling layer to reduce spatial dimensions
model.add(GlobalAveragePooling2D())

# Add a dense layer with 256 units and ReLU activation
model.add(Dense(256, activation='relu'))

# Add the output layer with softmax activation for classification
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Print a summary of the model
model.summary()

# Train the model
history = model.fit(train_images, train_labels, epochs=52, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_acc}')


Expression Images Shape: (1083, 224, 224, 3)
Expression Labels Shape: (1083,)
87910968/87910968 [==============================] - 1s 0us/step


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
 dense_1 (Dense)             (None, 7)                 1799      
                                                                 
Total params: 22329127 (85.18 MB)
Trainable params: 526343 (2.01 MB)
Non-trainable params: 21802784 (83.17 MB)
_________________________________________________________________
Epoch 1/52
22/22 [==============================] - 137s 6s/step - loss: 23.1

In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# Convert lists to NumPy arrays
expression_images = np.array(expression_images)
expression_labels = np.array(expression_labels)

# Check the shape of the image data and labels
print("Expression Images Shape:", expression_images.shape)
print("Expression Labels Shape:", expression_labels.shape)

# Obtain the number of class labels
num_classes = len(np.unique(expression_labels))

# Convert labels to one-hot encoding
expression_labels_one_hot = to_categorical(expression_labels, num_classes=num_classes)

# Stratified Splitting of data into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(
    expression_images, expression_labels_one_hot, test_size=0.2, stratify=expression_labels, random_state=42
)

# Load the InceptionV3 model pre-trained on ImageNet data
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Create a sequential model
model = Sequential()

# Add the InceptionV3 base model
model.add(base_model)

# Add a global average pooling layer to reduce spatial dimensions
model.add(GlobalAveragePooling2D())

# Add a dense layer with 256 units and ReLU activation
model.add(Dense(256, activation='relu'))

# Add the output layer with softmax activation for classification
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Print a summary of the model
model.summary()


Expression Images Shape: (1083, 224, 224, 3)
Expression Labels Shape: (1083,)
87910968/87910968 [==============================] - 1s 0us/step


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
 dense_1 (Dense)             (None, 7)                 1799      
                                                                 
Total params: 22329127 (85.18 MB)
Trainable params: 526343 (2.01 MB)
Non-trainable params: 21802784 (83.17 MB)
_________________________________________________________________


In [ ]:
import tensorflow as tf

model.save('/content/drive/My Drive/GAN_Models/GoogleNet_Microexp.h5')
loaded_model = tf.keras.models.load_model('/content/drive/My Drive/GAN_Models/GoogleNet_Microexp.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [6]:
import tensorflow as tf
model = tf.keras.models.load_model('/content/drive/My Drive/GAN_Models/GoogleNet_Microexp.h5')
from sklearn.metrics import precision_score, recall_score, f1_score, balanced_accuracy_score

# ... (previous code for model training and evaluation)

# Get predicted labels
predicted_probabilities = model.predict(test_images)
predicted_labels = np.argmax(predicted_probabilities, axis=1)

# Convert one-hot encoded test labels back to categorical labels
true_labels = np.argmax(test_labels, axis=1)
test_loss, test_acc = model.evaluate(test_images, test_labels)
# Print accuracy, precision, recall, F1-score, and UAR
print(f'Test Accuracy: {test_acc:.4f}')
print(f'Precision: {precision_score(true_labels, predicted_labels, average="weighted"):.4f}')
print(f'Recall: {recall_score(true_labels, predicted_labels, average="weighted"):.4f}')
print(f'F1-score: {f1_score(true_labels, predicted_labels, average="weighted"):.4f}')
print(f'Unweighted Average Recall (UAR): {balanced_accuracy_score(true_labels, predicted_labels):.4f}')

7/7 [==============================] - 32s 4s/step - loss: 0.6972 - accuracy: 0.8295
Test Accuracy: 0.8295
Precision: 0.8464
Recall: 0.8295
F1-score: 0.8314
Unweighted Average Recall (UAR): 0.8277
